In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

┌ Info: Precompiling GraphPPL [b3f8163a-e979-4e85-b43e-1f63d8c8b42c]
└ @ Base loading.jl:1278
┌ Warning: Package GraphPPL does not have ReactiveMP in its dependencies:
│ - If you have GraphPPL checked out for development and have
│   added ReactiveMP as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with GraphPPL
└ Loading ReactiveMP into GraphPPL from project dependency, future warnings for GraphPPL are suppressed.


In [2]:
@model function smoothing(n, x0, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    c = constvar(1.0)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
@rule NormalMeanVariance(:μ, Marginalisation) (m_out::Any, m_v::Missing) = missing
@rule NormalMeanVariance(:μ, Marginalisation) (m_out::Missing, m_v::Any) = missing

@rule typeof(+)(:in1, Marginalisation) (m_out::Missing, m_in2::Any) = missing
@rule typeof(+)(:in1, Marginalisation) (m_out::Any, m_in2::Missing) = missing

In [4]:
P = 1.0

n = 500
data = convert(Vector{Union{Float64, Missing}}, collect(1:n) + rand(Normal(0.0, sqrt(P)), n));

for index in map(d -> rem(abs(d), n), rand(Int, Int(n / 2)))
    data[index] = missing
end

In [7]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, P);

    buffer    = Vector{Marginal}(undef, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, (ms) -> copyto!(buffer, ms))
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return buffer
end

inference (generic function with 1 method)

In [8]:
x0_prior = NormalMeanVariance(0.0, 1000.0)
res = inference(data, x0_prior, P)

500-element Array{Marginal,1}:
 Marginal(NormalMeanVariance{Float64}(μ=1.056838217035798, v=0.0032362354814385753))
 Marginal(NormalMeanVariance{Float64}(μ=2.0568382170357977, v=0.003236235481438575))
 Marginal(NormalMeanVariance{Float64}(μ=3.0568382170357977, v=0.003236235481438575))
 Marginal(NormalMeanVariance{Float64}(μ=4.056838217035799, v=0.0032362354814385753))
 Marginal(NormalMeanVariance{Float64}(μ=5.056838217035797, v=0.003236235481438575))
 Marginal(NormalMeanVariance{Float64}(μ=6.056838217035797, v=0.003236235481438575))
 Marginal(NormalMeanVariance{Float64}(μ=7.056838217035798, v=0.003236235481438575))
 Marginal(NormalMeanVariance{Float64}(μ=8.056838217035798, v=0.003236235481438575))
 Marginal(NormalMeanVariance{Float64}(μ=9.056838217035798, v=0.003236235481438575))
 Marginal(NormalMeanVariance{Float64}(μ=10.056838217035798, v=0.003236235481438575))
 Marginal(NormalMeanVariance{Float64}(μ=11.056838217035798, v=0.0032362354814385744))
 Marginal(NormalMeanVariance{Float64}(